In [ ]:
!pip install gtts


In [ ]:
!pip install scipy
!pip install translate
!pip install ipywidgets
!pip install vosk
!pip install pyaudio
!pip install pandas
!pip install numpy
!pip install keras
!pip install tensorflow



In [ ]:
# Install pyaudio from http://people.csail.mit.edu/hubert/pyaudio/
# Find audio device index using this code
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (2- USB Audio Device', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microphone (DroidCam Virtual Au', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputChann

In [ ]:
import pyaudio
p = pyaudio.PyAudio()
print(p.get_device_count())
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))

24
{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Microphone (2- USB Audio Device', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Microphone (DroidCam Virtual Au', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputCh

In [ ]:
import pyaudio
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxOutputChannels')) > 0:
            print("Output Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))


Output Device id  3  -  Microsoft Sound Mapper - Output
Output Device id  4  -  Speakers (2- USB Audio Device)
Output Device id  5  -  Digital Audio (S/PDIF) (High De


In [ ]:
!pip install google.cloud

In [ ]:
!pip install pyttsx3
!pip install gtts

In [ ]:
###import pyaudio
###def callback(in_data, frame_count, time_info, status):
###    print("Received audio data")
###    return (in_data, pyaudio.paContinue)
###p = pyaudio.PyAudio()
###stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=16000, stream_callback=callback)
###stream.start_stream()


In [ ]:
import pyaudio
import threading
import vosk
import ipywidgets as widgets
import json
import os
import subprocess
import time
import scipy.signal
from translate import Translator
from gtts import gTTS
from IPython.display import Audio
import io
import pyttsx3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

# Dictionary for Vosk model names
vosk_model_names = {
    "es": "C:\\Users\\jenny\\Notebooks\\vosk-model-small-es-0.42"
}
english_keras_model = load_model("C:\\Users\\jenny\\Notebooks\\TrainedData\\trained_model.h5")


def setup_recognizer(language):
    if language == "en":
        # Use Keras model for English
        return english_keras_model
    else:
        model_name = vosk_model_names[language]
        model = vosk.Model(model_name)
        recognizer = vosk.Recognizer(model, 16000)
        recognizer.SetWords(True)
        return recognizer

rec = setup_recognizer("en")

record_button = widgets.Button(description="Record")
stop_button = widgets.Button(description="Stop")


# Define the audio stream callback function
def callback(in_data, frame_count, time_info, status):
    if status:
        print(status, file=sys.stderr)
    # Convert the audio data to the format expected by Vosk
    if in_data:
        if len(in_data) % 2 == 1:
            in_data = in_data[:-1]
        rec.AcceptWaveform(in_data)
    return (in_data, pyaudio.paContinue)

p = pyaudio.PyAudio()


def on_record_clicked(b):
    global stream, rec
    rec = setup_recognizer(recognition_dropdown.value)
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=16000, stream_callback=callback)
    stream.start_stream()

def on_stop_clicked(b):
    global p, stream, translated_text
    stream.stop_stream()
    stream.close()
    p.terminate()
    text = rec.FinalResult()
    result = json.loads(text)
    print("Result:", result["text"])
    if result["text"]:
        try:
            translated_text = translate_text(result["text"], dropdown.value)
            print(f"Translated text ({dropdown.value}): {translated_text}")
        except AttributeError as e:
            print("Error:", str(e))

record_button.on_click(on_record_clicked)
stop_button.on_click(on_stop_clicked)

# Define the translation function
def translate_text(text, target_language):
    try:
        translator = Translator(to_lang=target_language)
        translation = translator.translate(text)
        return translation
    except Exception as e:
        print("Error:", str(e))
        return ""

# Define the GUI elements
dropdown = widgets.Dropdown(options=[("English", "en"), ("Spanish", "es")], description="Language:")
output_box = widgets.Output()


# Define the GUI elements
play_button = widgets.Button(description="Play Translation")
audio_output = widgets.Output()

# Define the click event function for the play button
def on_play_button_clicked(b):
    with audio_output:
        global translated_text
        if translated_text:
            tts = gTTS(text=translated_text, lang=dropdown.value)
            with io.BytesIO() as f:
                tts.write_to_fp(f)
                f.seek(0)
                display(Audio(f.read(), autoplay=True))
        else:
            print("Please translate the text first.")

# Speech recognition language selection
recognition_dropdown = widgets.Dropdown(options=[("English", "en"), ("Spanish", "es")], description="Recognize:")
display(recognition_dropdown)

# Text-to-speech engine setup
tts_engine = pyttsx3.init()
voices = tts_engine.getProperty('voices')
voice_dropdown = widgets.Dropdown(options=[(voice.name, voice.id) for voice in voices], description="Voice:")
display(voice_dropdown)

# Attach the event function to the play button
play_button.on_click(on_play_button_clicked)

# Improved layout
input_widgets = widgets.VBox([widgets.HBox([recognition_dropdown, dropdown, voice_dropdown]), widgets.HBox([record_button, stop_button, play_button])])
display(input_widgets)
display(output_box)
display(audio_output)



Dropdown(description='Recognize:', options=(('English', 'en'), ('French', 'fr'), ('Spanish', 'es'), ('Japanese…

Dropdown(description='Voice:', options=(('Microsoft David Desktop - English (United States)', 'HKEY_LOCAL_MACH…

Output()

Output()